## Part 4: Topic Modelling on Clinical Notes
### Daniel Dastoor
### Part a:

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from gensim.models import LdaMulticore, LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import datapath
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string, re
import multiprocessing as mp

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
dataset = pd.read_csv("~/csc2548_ml4h/adult_notes.gz", compression='gzip')
dataset = dataset[~pd.isnull(dataset.chartext)]

chartext = dataset.chartext

In [ ]:
stopwordList = stopwords.words("english")

# mdeicalStopwordList adapted from
# https://www.cs.stanford.edu/people/sonal/gupta14jamia_supl.pdf
medicalStopwordList = ["patient", "patients", "disease", "diseases", "disorder", "symptom", "symptoms", "drug", "drugs", "problems", "problem", "prob", "probs", "med", "meds", "pill", "pills", "medicine", "medicines", "medication", "medications", "treatment", "treatments", "caps", "capsules", "capsule", "tablet", "tablets", "tabs", "doctor", "dr", "dr.", "doc", "physician", "physicians", "test", "tests", "testing", "specialist", "specialists", "side-effect", "side-effects", "pharmaceutical", "pharmaceuticals", "pharma", "diagnosis", "diagnose", "diagnosed", "exam", "challenge", "device", "condition", "conditions", "suffer", "suffering", "suffered", "feel", "feeling", "prescription", "prescribe", "prescribed", "over-the-counter", "otc"]

units = ["ml", "mgdl", "pt", "pm", "meql"]

stopwordList = stopwordList + medicalStopwordList + units

def tokenize(s):
    return [j for j in [re.sub("^\d$", '', re.sub("[\W_]+|^\d$", '', i)) for i in word_tokenize(s.lower())] 
            if j not in stopwordList and j != '']

pool = mp.Pool(mp.cpu_count())
corpus = pool.map(tokenize, chartext)
pool.close()
pool.join()

#### Comment on preprocessing:

Additional steps taken:

1. Tokens of type of single digit numbers were removed, as they are not contributing to the topics when out of context.

2. Medical domain "stopwords" were added to the list of stopwords, to remove words used too frequently and improve the relevance of topics and the words in them.

2. Units of measure "stopwords" were added to the list of stopwords, to remove words used too frequently and improve the relevance of topics and the words in them.

In [8]:
dictionary = Dictionary(corpus)

In [9]:
pool = mp.Pool(mp.cpu_count())
bow_corpus = pool.map(dictionary.doc2bow, corpus)
pool.close()
pool.join()
#del chartext
#del corpus

In [11]:
def generate_lda(topics: int) -> LdaMulticore:
    lda = LdaModel(bow_corpus, num_topics=topics, id2word=dictionary)

    cm = CoherenceModel(model=lda, corpus=bow_corpus, dictionary=dictionary, coherence="u_mass")
    score = cm.get_coherence()
    
    print(f"LDA: # of topics: {topics}, Coherence score: {score}\n")
    
    lda.save("lda_"+str(topics))
    
    return lda, score

lda_20, score_20 = generate_lda(20)
lda_50, score_50 = generate_lda(50)
lda_100, score_100 = generate_lda(100)

LDA: # of topics: 20, Coherence score: -0.6053154561330254

LDA: # of topics: 50, Coherence score: -0.8122645424338729

LDA: # of topics: 100, Coherence score: -1.2643940704601395



 #### How many topics gives the optimal coherence score? 
 
 The LDA model generated on 20 topics gave the optimal coherence score.


### Part b:

In [12]:
lda = lda_20

def examine_word(word: str, model: LdaMulticore):
    topics = model.get_document_topics(dictionary.doc2bow([word]), per_word_topics=True)[1][0][1]
    print(f"\nWord \"{word}\" found in topics {topics}")
    for topic in topics:
        print(f"Topic {topic}:")
        print(model.show_topic(topic, topn=20))
    
examine_word("respiratory", lda)


Word "respiratory" found in topics [12]
Topic 12:
[('trach', 0.03310057), ('tracheal', 0.026437256), ('stent', 0.023169182), ('tracheobronchomalacia', 0.019722452), ('tbm', 0.015810193), ('ip', 0.012208484), ('bronch', 0.012193589), ('tracheomalacia', 0.011539162), ('tracheostomy', 0.010841105), ('bronchomalacia', 0.008184757), ('bronchoscopy', 0.0075094975), ('respiratory', 0.0069821076), ('copd', 0.0063179596), ('rigid', 0.0056467513), ('granulation', 0.0047931783), ('assessment', 0.004592882), ('tracheobronchial', 0.0042763017), ('anxiety', 0.0033587394), ('malacia', 0.0033555909), ('airway', 0.0032051068)]


The other words in this topic do make sense, as words like trach, tracheal, stent, tracheobronchomalacia, bronch, airway, and etc, are all in the area of respiratory issues.
As such, the name for this topic would be **Respiratory Issues**.

In [13]:
examine_word("vomiting", lda)


Word "vomiting" found in topics [0]
Topic 0:
[('pain', 0.015115606), ('name', 0.008588985), ('w', 0.0059262337), ('iv', 0.0058275103), ('co', 0.0055646095), ('hr', 0.0054285373), ('given', 0.0052032745), ('plan', 0.005185598), ('po', 0.004885575), ('monitor', 0.004839208), ('bp', 0.0048359516), ('last', 0.0046347715), ('hct', 0.0045250305), ('clear', 0.0044288193), ('neuro', 0.004040973), ('stable', 0.0038911263), ('oriented', 0.0035876993), ('urine', 0.0034098094), ('ni', 0.003396138), ('per', 0.003368731)]


The words in this topic are more general, and less specific to the word vomiting, however the word vomiting iteslf is more general and can occur in many situations. As such, it is not possible to give a name for this topic.

In [14]:
examine_word("urine", lda)


Word "urine" found in topics [15]
Topic 15:
[('name', 0.007635522), ('vent', 0.0061960984), ('secretions', 0.005555829), ('resp', 0.005498929), ('intubated', 0.005368761), ('plan', 0.0052429694), ('care', 0.005150099), ('abg', 0.004983796), ('noted', 0.004880459), ('last', 0.004415221), ('remains', 0.0041969502), ('thick', 0.004180449), ('clear', 0.0041589546), ('given', 0.004072302), ('neuro', 0.0039990884), ('hr', 0.0038723384), ('propofol', 0.003761605), ('sputum', 0.003690738), ('bp', 0.003616769), ('placed', 0.0035625577)]


The words in this topic do make sense, as words like secretions, last, thick, and clear, can be related to urine. As such, the name for this topic would be **Fluid Issues**.

In [15]:
examine_word("pulse", lda)


Word "pulse" found in topics [3]
Topic 3:
[('plan', 0.006358578), ('assessment', 0.0060812323), ('icu', 0.005287636), ('history', 0.0052462295), ('continue', 0.004929864), ('given', 0.0047584246), ('assessed', 0.0046859207), ('pain', 0.004636236), ('last', 0.0046199583), ('response', 0.004450188), ('name', 0.0042789257), ('po', 0.0041863313), ('action', 0.004176705), ('total', 0.0038810014), ('labs', 0.003755706), ('c', 0.0036512334), ('respiratory', 0.0035958285), ('24', 0.0035546946), ('code', 0.0035232862), ('blood', 0.0034580214)]


The other words in this topic do make sense, as words like icu, assessed, last, response, code, and blood can be related to pulse. As such, the name for this topic would be **Critical**.